📈 Entendendo o Código do Simulador de Desempenho de Ativos Financeiros
Este Jupyter Notebook tem como objetivo detalhar as principais seções do código-fonte do aplicativo Streamlit (app.py), que simula o desempenho de ativos financeiros na bolsa de valores.

Ele aborda desde o carregamento dos dados até a visualização dinâmica e a simulação de estratégias de trade.

#1. Configuração Inicial e Importações
O aplicativo Streamlit começa com importações de bibliotecas e configurações básicas da página.

In [ ]:
import streamlit as st
import pandas as pd
import numpy as np
import time
import plotly.graph_objects as go
from datetime import date, timedelta
import calendar

# Configuração da página Streamlit
st.set_page_config(layout="wide", page_title="Simulação Dinâmica de Ativos")

# Títulos e descrições na interface do usuário
st.title("📈 Simulador de Desempenho de Ativos Financeiros")
st.write("Visualização de valores reais e previstos para ativos da Bolsa.")

* streamlit: Para construir a interface web interativa.

* pandas: Para manipulação de DataFrames (dados tabulares).

* numpy: Para operações numéricas, especialmente com arrays (.npy).

* time: Para introduzir atrasos na plotagem dinâmica.

* plotly.graph_objects: Para criar gráficos interativos.

* datetime, timedelta, calendar: Para manipulação de datas.

# 2. A Classe ActionPredictionTrading
Esta classe é o coração da lógica de simulação de trading. Ela foi adaptada para receber dados já preparados (preços reais e previstos para uma semana específica) e simular as operações de trade e a estratégia Buy-and-Hold.

Ela não carrega modelos nem faz previsões; ela apenas processa os resultados já obtidos.

In [ ]:
import numpy as np
import pandas as pd

class ActionPredictionTrading:
    """
    Classe para simular operações de trading e comparar com buy-and-hold,
    recebendo os valores reais e preditos para o período de simulação.
    """

    def __init__(self, simulation_df: pd.DataFrame, ticker: str):
        """
        Args:
            simulation_df: DataFrame já preparado com as colunas
                           ['date', 'actual', 'predicted', 'actual_next']
                           para a semana de simulação.
            ticker: O ticker da ação que está sendo simulada.
        """
        if not all(col in simulation_df.columns for col in ['date', 'actual', 'predicted', 'actual_next']):
            raise ValueError("O DataFrame de simulação deve conter as colunas 'date', 'actual', 'predicted', 'actual_next'.")

        self.df = simulation_df.copy()
        self.full_df = simulation_df.copy() # full_df também será o período da simulação para B&H
        self.ticker = ticker
        self.df.dropna(subset=['actual_next'], inplace=True)
        self.full_df.dropna(subset=['actual_next'], inplace=True)

    def simulate_trading(
        self,
        stop_loss: bool = False,
        initial_capital: float = 100000,
        shares_per_trade: int = 100,
        stop_type: str = 'percent',
        stop_value: float = 0.03,
        dead_zone_pct: float = 0.005
    ) -> dict:
        # Lógica de simulação de trades diários baseada na previsão
        # Calcula PnL, capital, taxa de acerto, Sharpe Ratio, Max Drawdown
        # ... (código completo da função simulate_trading) ...
        capital = initial_capital
        capital_history = [capital]
        hits = 0
        total_trades = 0
        profits = []
        stop_triggered = 0

        for i in range(len(self.df)):
            price_today = self.df.iloc[i]['actual']
            price_tomorrow = self.df.iloc[i]['actual_next']
            predicted_price = self.df.iloc[i]['predicted']

            limit = stop_value * price_today if stop_type == 'percent' else stop_value
            limit_amt = limit * shares_per_trade

            position = 'long' if predicted_price > price_today else 'short'
            pnl = ((price_tomorrow - price_today) if position == 'long'
                   else (price_today - price_tomorrow)) * shares_per_trade

            if stop_loss and pnl < -limit_amt:
                pnl = -limit_amt
                stop_triggered += 1

            capital += pnl
            profits.append(pnl)
            total_trades += 1
            if pnl > 0:
                hits += 1
            capital_history.append(capital)

        if total_trades == 0:
            return {
                'total_return': 0.0, 'hit_rate': 0.0, 'sharpe_ratio': 0.0,
                'max_drawdown': 0.0, 'final_capital': initial_capital,
                'total_trades': 0, 'stop_triggered': 0,
                'predicted_prices': [], 'today_prices': [], 'tomorrow_prices': [], 'dates': []
            }

        hit_rate = hits / total_trades if total_trades else 0
        total_return = (capital - initial_capital) / initial_capital
        sharpe_ratio = (np.mean(profits) / np.std(profits)
                        if len(profits) > 1 and np.std(profits) != 0 else 0)
        peak = np.maximum.accumulate(capital_history)
        max_drawdown = np.max((peak - capital_history) / peak)

        return {
            'total_return': total_return,
            'hit_rate': hit_rate,
            'sharpe_ratio': sharpe_ratio,
            'max_drawdown': max_drawdown,
            'final_capital': capital,
            'total_trades': total_trades,
            'stop_triggered': stop_triggered,
            'predicted_prices': self.df['predicted'].tolist(),
            'today_prices': self.df['actual'].tolist(),
            'tomorrow_prices': self.df['actual_next'].tolist(),
            'dates': self.df['date'].tolist()
        }

    def simulate_buy_and_hold(
        self,
        initial_capital: float = 100000,
        shares: int = 100
    ) -> dict:
        # Lógica de simulação da estratégia Buy-and-Hold
        # ... (código completo da função simulate_buy_and_hold) ...
        df_bh = self.full_df
        if df_bh.empty:
            raise ValueError("DataFrame for Buy-and-Hold is empty. Ensure the data was loaded correctly.")

        price_buy = df_bh.iloc[0]['actual']
        price_sell = df_bh.iloc[-1]['actual_next'] if not df_bh['actual_next'].isnull().all() else df_bh.iloc[-1]['actual']

        profit = (price_sell - price_buy) * shares
        final_capital = initial_capital + profit
        total_return = profit / initial_capital

        capital_history = [initial_capital]
        for i in range(len(df_bh)):
            current_capital = initial_capital + (df_bh.iloc[i]['actual'] - price_buy) * shares
            capital_history.append(current_capital)

        if len(df_bh) > 0 and 'actual_next' in df_bh.columns and not pd.isna(df_bh.iloc[-1]['actual_next']):
             capital_history.append(initial_capital + (df_bh.iloc[-1]['actual_next'] - price_buy) * shares)

        return {
            'total_return': total_return,
            'initial_price': price_buy,
            'final_price': price_sell,
            'final_capital': final_capital,
            'shares_held': shares,
            'days_held': len(df_bh) + 1,
            'capital_history': capital_history
        }


# 3. Funções de Carregamento de Dados
Estas funções são responsáveis por carregar os dados históricos (stocks.csv) e os resultados pré-calculados do conjunto de teste (.npy). Elas utilizam o @st.cache_data para otimizar o desempenho, garantindo que os dados sejam carregados apenas uma vez.

In [ ]:
@st.cache_data
def load_all_stock_data_simplified():
    """
    Carrega o DataFrame completo onde cada coluna é uma ação.
    A primeira coluna é a 'Date'.
    """
    try:
        df = pd.read_csv('stocks.csv', parse_dates=['Date'], index_col='Date')
        df = df.sort_index() # Garante que as datas estão ordenadas
        return df
    except FileNotFoundError:
        st.error("Arquivo 'stocks.csv' não encontrado. Por favor, coloque-o na mesma pasta do script.")
        st.stop()
    except Exception as e:
        st.error(f"Erro ao carregar stocks.csv: {e}")
        st.stop()

@st.cache_data
def load_full_test_set_npy(asset_ticker):
    """Carrega os arquivos NPY para o conjunto de teste completo do ativo."""
    real_path = f'./data_npy/{asset_ticker}_real_test_set.npy'
    predicted_path = f'./data_npy/{asset_ticker}_predicted_test_set.npy'
    try:
        real_data = np.load(real_path)
        predicted_data = np.load(predicted_path)
        if len(real_data) != len(predicted_data):
            st.error(f"Erro: Os arquivos NPY para {asset_ticker} têm tamanhos diferentes. Real: {len(real_data)}, Predito: {len(predicted_data)}.")
            st.stop()
        return real_data, predicted_data
    except FileNotFoundError:
        st.error(f"Arquivos NPY de conjunto de teste para {asset_ticker} não encontrados. Esperado: '{real_path}' e '{predicted_path}'.")
        st.error("Por favor, verifique se seus arquivos NPY estão nomeados corretamente e na pasta './data_npy/'.")
        st.stop()
    except Exception as e:
        st.error(f"Erro ao carregar arquivos NPY para {asset_ticker}: {e}")
        st.stop()

* load_all_stock_data_simplified(): Lê o stocks.csv, define a coluna 'Date' como índice e garante a ordenação por data.

* load_full_test_set_npy(): Carrega os arrays NumPy (.npy) contendo os valores reais e previstos do conjunto de teste completo para uma ação específica. Ele espera que esses arquivos estejam na pasta data_npy/ e sigam a convenção de nomenclatura.

# 4. Função de Extração de Datas da Semana
Esta função é crucial para mapear a seleção do usuário (ano, mês, semana) para as 5 datas de trade reais presentes no seu stocks.csv.

In [ ]:
def get_five_trading_days_for_week(df_asset_index, year, month_name, week_number_str):
    """
    Retorna uma lista de 5 pd.Timestamps (segunda a sexta) para a semana selecionada
    a partir do índice de datas do DataFrame de um ativo.
    """
    month_map = {
        "Janeiro": 1, "Fevereiro": 2, "Março": 3, "Abril": 4, "Maio": 5, "Junho": 6,
        "Julho": 7, "Agosto": 8, "Setembro": 9, "Outubro": 10, "Novembro": 11, "Dezembro": 12
    }
    month_num = month_map[month_name]
    week_num = int(week_number_str.split('ª')[0])

    dates_in_month_timestamps = df_asset_index[(df_asset_index.year == year) &
                                                 (df_asset_index.month == month_num)].tolist()
    dates_in_month_timestamps.sort()

    if not dates_in_month_timestamps:
        return None, "Não há dados para o mês e ano selecionados no histórico disponível."

    # Agrupar datas em "semanas" de 5 dias úteis
    weeks_list_of_timestamps = [dates_in_month_timestamps[i:i + 5] for i in range(0, len(dates_in_month_timestamps), 5)]

    if week_num > 0 and week_num <= len(weeks_list_of_timestamps):
        selected_week_timestamps = weeks_list_of_timestamps[week_num - 1]

        # Validação crítica: a semana deve ter exatamente 5 dias úteis
        if len(selected_week_timestamps) != 5:
            return None, f"A {week_number_str} de {month_name} em {year} tem {len(selected_week_timestamps)} dias negociados, mas esperamos 5 para a simulação de trading. Escolha outra semana."

        return selected_week_timestamps, None
    else:
        return None, "Semana selecionada fora do intervalo de dados disponíveis para o mês."


* Esta função filtra as datas do índice do DataFrame para o ano e mês selecionados.

* Ela então agrupa essas datas em blocos de 5, representando as semanas de trade.

* A validação len(selected_week_timestamps) != 5 é crucial para garantir que a simulação ocorra apenas para semanas completas de 5 dias úteis.

# 5. Interface do Usuário (Sidebar)
A barra lateral do Streamlit (st.sidebar) permite que o usuário selecione a ação, o ano, o mês e a semana para a simulação, além de configurar os parâmetros da estratégia de trade.

In [ ]:
# --- Sidebar para Seleção ---
st.sidebar.header("Configurações da Simulação")

available_assets = df_all_stocks.columns.tolist()
selected_asset = st.sidebar.selectbox("Selecione o Ativo:", available_assets)

df_selected_asset_series = df_all_stocks[selected_asset]
available_years = sorted(df_selected_asset_series.index.year.unique().tolist(), reverse=True)
selected_year = st.sidebar.selectbox("Selecione o Ano:", available_years)

available_months = ["Janeiro", "Fevereiro", "Março", "Abril", "Maio", "Junho", "Julho", "Agosto", "Setembro", "Outubro", "Novembro", "Dezembro"]
selected_month = st.sidebar.selectbox("Selecione o Mês:", available_months)

available_weeks = ["1ª Semana", "2ª Semana", "3ª Semana", "4ª Semana", "5ª Semana"]
selected_week = st.sidebar.selectbox("Selecione a Semana:", available_weeks)

# Opções de simulação de trading
st.sidebar.subheader("Parâmetros da Estratégia")
initial_capital = st.sidebar.number_input("Capital Inicial:", min_value=1000.0, value=100000.0, step=1000.0)
shares_per_trade = st.sidebar.number_input("Ações por Trade:", min_value=1, value=100, step=10)
enable_stop_loss = st.sidebar.checkbox("Habilitar Stop Loss?", value=True)
stop_value_percent = st.sidebar.slider("Valor Stop Loss (%)", min_value=0.5, max_value=10.0, value=3.0, step=0.5) / 100.0

* st.selectbox: Cria menus suspensos para seleção de ativo, ano, mês e semana.

* st.number_input, st.slider, st.checkbox: Permitem ao usuário configurar os parâmetros da simulação de trade.

# 6. Lógica Principal da Simulação
Este é o bloco de código que é executado quando o botão "Rodar Simulação" é clicado. Ele orquestra o carregamento de dados, a preparação para a classe de trading e a execução das simulações.

In [ ]:
if st.sidebar.button("Rodar Simulação"):
    st.info(f"Preparando simulação para **{selected_asset}** na **{selected_week}** de **{selected_month}/{selected_year}**...")

    # 1. Carregar os conjuntos de teste completos para o ativo selecionado
    real_full_test, predicted_full_test = load_full_test_set_npy(selected_asset)

    # 2. Obter as 5 datas correspondentes à semana selecionada para o ativo específico
    five_trading_days_timestamps, error_msg = \
        get_five_trading_days_for_week(df_selected_asset_series.index, selected_year, selected_month, selected_week)

    if error_msg:
        st.error(error_msg)
        st.stop()

    # 3. Encontrar a fatia correta nos arrays NPY usando as datas
    num_test_points = len(real_full_test)
    test_set_dates_in_df = df_selected_asset_series.index[-num_test_points:]

    try:
        start_idx_in_test_set = test_set_dates_in_df.get_loc(five_trading_days_timestamps[0])
    except KeyError:
        st.error(f"Erro: A primeira data da semana selecionada ({five_trading_days_timestamps[0].strftime('%Y-%m-%d')}) não foi encontrada no período de teste NPY para {selected_asset}. Verifique a consistência das datas e do conjunto de teste.")
        st.stop()

    if (start_idx_in_test_set + 5) > num_test_points:
        st.error(f"Dados NPY insuficientes para a semana selecionada. O conjunto de teste NPY não cobre o período até {five_trading_days_timestamps[-1].strftime('%Y-%m-%d')}.")
        st.stop()

    real_values_for_plot = real_full_test[start_idx_in_test_set : start_idx_in_test_set + 5]
    predicted_values_for_plot = predicted_full_test[start_idx_in_test_set : start_idx_in_test_set + 5]

    dates_for_plot = five_trading_days_timestamps

    if not (len(dates_for_plot) == len(real_values_for_plot) == len(predicted_values_for_plot) == 5):
        st.error("Erro interno: Inconsistência no número de pontos para plotagem. Deveriam ser 5 para a semana de 5 dias.")
        st.stop()

    # --- Preparar DataFrame para a Classe de Trading ---
    # Este DataFrame terá 4 linhas (para 4 trades: seg, ter, qua, qui)
    # A previsão de sexta é usada na decisão de quinta para sexta.
    df_for_trading_class = pd.DataFrame({
        'date': dates_for_plot[0:4],
        'actual': real_values_for_plot[0:4],
        'predicted': predicted_values_for_plot[1:5], # Previsão para o dia seguinte
        'actual_next': real_values_for_plot[1:5] # Preço real do dia seguinte
    })

    if df_for_trading_class.empty:
        st.warning("Nenhum dado de trade preparado para a semana selecionada. Certifique-se de que a semana tem pelo menos 5 dias úteis e dados NPY correspondentes.")
        st.stop()

    # --- Executar Simulações de Trading ---
    trading_simulator = ActionPredictionTrading(df_for_trading_class, ticker=selected_asset)

    model_strategy_results = trading_simulator.simulate_trading(
        stop_loss=enable_stop_loss,
        initial_capital=initial_capital,
        shares_per_trade=shares_per_trade,
        stop_value=stop_value_percent,
        stop_type='percent'
    )

    buy_and_hold_results = trading_simulator.simulate_buy_and_hold(
        initial_capital=initial_capital,
        shares=shares_per_trade
    )


* Carregamento e Fatiamento de NPYs: Os arrays .npy completos são carregados, e a fatia correspondente à semana de 5 dias é extraída com base nas datas.

* df_for_trading_class: Este DataFrame é crucial. Ele é construído com 4 linhas, onde cada linha representa um dia de trade (Segunda a Quinta). Para cada dia, ele contém:

  * date: A data do dia atual.

  * actual: O preço de fechamento do dia atual (preço de "hoje").

  * predicted: A previsão do modelo para o dia seguinte (previsão para "amanhã", feita "hoje").

  * actual_next: O preço real de fechamento do dia seguinte (preço real de "amanhã", para calcular o PnL).

* Execução das Simulações: As funções simulate_trading e simulate_buy_and_hold da classe ActionPredictionTrading são chamadas com os parâmetros definidos pelo usuário.

# 7. Plotagem Dinâmica do Gráfico
Esta seção é responsável por exibir o gráfico de preços reais vs. previstos de forma dinâmica, adicionando um ponto por vez.

In [ ]:
 # --- Início da Plotagem Dinâmica (Gráfico) ---
    st.subheader(f"Simulação Dinâmica para {selected_asset}")

    initial_data_display = pd.DataFrame({
        'Data': pd.to_datetime([]),
        'Preço Real': [],
        'Preço Previsto': []
    })

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=[], y=[], mode='lines+markers', name='Preço Real', line=dict(color='blue', width=2)))
    fig.add_trace(go.Scatter(x=[], y=[], mode='lines+markers', name='Preço Previsto', line=dict(color='red', dash='dot', width=2)))

    fig.update_layout(
        title='Preços Reais vs. Preços Previstos (Simulação Semanal)',
        xaxis_title='Data',
        yaxis_title='Preço de Fechamento',
        hovermode="x unified",
        legend=dict(x=0.01, y=0.99, bordercolor="Black", borderwidth=1),
        xaxis=dict(
            tickformat='%d/%m/%Y',
            automargin=True
        )
    )

    chart_placeholder = st.empty()
    table_placeholder = st.empty()

    st.info("Simulação de gráfico em andamento... Aguarde os pontos serem plotados. ⏳")

    display_df = initial_data_display.copy()

    for i in range(len(dates_for_plot)): # Itera sobre os 5 dias para o gráfico
        new_row = pd.DataFrame({
            'Data': [dates_for_plot[i]],
            'Preço Real': [real_values_for_plot[i]],
            'Preço Previsto': [predicted_values_for_plot[i]]
        })
        display_df = pd.concat([display_df, new_row], ignore_index=True)

        with fig.batch_update():
            fig.data[0].x = display_df['Data']
            fig.data[0].y = display_df['Preço Real']
            fig.data[1].x = display_df['Data']
            fig.data[1].y = display_df['Preço Previsto']

            # Força o Plotly a usar apenas as datas que temos como ticks
            fig.update_xaxes(
                tickmode='array',
                tickvals=display_df['Data'].tolist(),
                ticktext=[d.strftime('%d/%m') for d in display_df['Data']]
            )

        with chart_placeholder:
            st.plotly_chart(fig, use_container_width=True)

        with table_placeholder:
            st.dataframe(display_df.set_index('Data').style.format(precision=2))

        time.sleep(0.7)

    st.success("Simulação de gráfico concluída! ✅")

* st.empty(): Cria placeholders para o gráfico e a tabela, permitindo que sejam atualizados no mesmo local.

* go.Figure() e fig.batch_update(): Usados para criar e atualizar o gráfico Plotly de forma eficiente.

* Loop for i in range(len(dates_for_plot)): Itera sobre os 5 dias da semana, adicionando um ponto por vez ao gráfico e à tabela.

* time.sleep(0.7): Introduz um atraso para criar o efeito de "tempo real".

* fig.update_xaxes(tickmode='array', ...): Garante que o eixo X do gráfico exiba apenas as 5 datas relevantes, sem repetições.

# 8. Exibição dos Resultados das Estratégias
Após a simulação gráfica, uma tabela comparativa é exibida, mostrando as métricas de desempenho para a estratégia baseada no modelo e para a estratégia Buy-and-Hold.

In [ ]:
  # --- Exibir Resultados das Estratégias (Tabela) ---
    st.subheader("📊 Comparativo de Estratégias de Trade")

    results_data = {
        "Métrica": [
            "Retorno Total",
            "Capital Final",
            "Taxa de Acerto (Hit Rate)",
            "Sharpe Ratio",
            "Max Drawdown",
            "Total de Trades",
            "Stop Loss Acionado"
        ],
        "Estratégia do Modelo": [
            f"{model_strategy_results['total_return']:.2%}",
            f"R$ {model_strategy_results['final_capital']:,.2f}",
            f"{model_strategy_results['hit_rate']:.2%}",
            f"{model_strategy_results['sharpe_ratio']:.2f}",
            f"{model_strategy_results['max_drawdown']:.2%}",
            f"{model_strategy_results['total_trades']}",
            f"{model_strategy_results['stop_triggered']}"
        ],
        "Buy-and-Hold": [
            f"{buy_and_hold_results['total_return']:.2%}",
            f"R$ {buy_and_hold_results['final_capital']:,.2f}",
            "-",
            "-",
            "-",
            "-",
            "-"
        ]
    }

    df_results = pd.DataFrame(results_data)
    st.table(df_results.set_index("Métrica"))

    st.markdown("""
    **Observações:**
    * **Retorno Total:** Lucro/Prejuízo percentual em relação ao capital inicial.
    * **Capital Final:** Valor final do capital após a simulação.
    * **Taxa de Acerto (Hit Rate):** Percentual de trades lucrativos.
    * **Sharpe Ratio:** Mede o retorno da estratégia ajustado ao risco. Valores maiores são melhores.
    * **Max Drawdown:** Maior queda percentual do capital a partir de um pico.
    """)



* Os resultados das simulações são formatados e apresentados em um pd.DataFrame, que é então exibido como uma tabela interativa usando st.table().

* Explicações adicionais sobre as métricas são fornecidas para clareza.

# Conclusão
Este notebook detalhou as principais componentes do seu aplicativo Streamlit, desde o carregamento e preparação de dados até a simulação de estratégias de trading e a visualização dinâmica dos resultados. A modularidade do código e o uso de funções de cache do Streamlit contribuem para um aplicativo eficiente e fácil de manter.